In [1]:
import os
import sys

import logging
import re
import pathlib

import datetime

import numpy as np
import pandas as pd
from scipy.stats import pearsonr

import matplotlib.pyplot as plt

from pyplink import PyPlink

from basic_tools import *

In [2]:
import glob

In [4]:
plink_KCHIP_HLA_AA_SNP_1000G=PyPlink(plink_KCHIP_HLA_AA_SNP_1000G_path)
plink_KCHIP_HLA_AA_SNP_1000G_fam=plink_KCHIP_HLA_AA_SNP_1000G.get_fam().astype({'fid':str,'iid':str}).rename(columns={'fid':'FID','iid':'IID'})
plink_KCHIP_HLA_AA_SNP_1000G_bim=plink_KCHIP_HLA_AA_SNP_1000G.get_bim()

In [5]:
gene_bed_path='data/known_genes_chr6.hg18.txt'
gene_bed=pd.read_csv(gene_bed_path,sep='\t')
gene_bed=gene_bed[(gene_bed['txStart']>=plink_KCHIP_HLA_AA_SNP_1000G_bim.pos.min())&(gene_bed['txEnd']<=plink_KCHIP_HLA_AA_SNP_1000G_bim.pos.max())]

gene_assign=plink_KCHIP_HLA_AA_SNP_1000G_bim[['pos']]

for idx,row in gene_bed.iterrows():
    gene_assign[row['name2']]=0
    
for idx,row in gene_bed.iterrows():    
    gene_assign[row['name2']][(gene_assign['pos']>=row['txStart'])&(gene_assign['pos']<=row['txEnd'])]=1
    
HLA_names=np.unique([i[0].split('_')[1] for i in plink_KCHIP_HLA_AA_SNP_1000G_bim[plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains('HLA_')].index.str.split('*')])

for HLA_name in HLA_names:
    #print(HLA_name)
    HLA_name='HLA-{}'.format(HLA_name)
    #gene_assign_HLA[HLA_name]=0
    gene_select=gene_assign[gene_assign.index.str.contains('HLA_'+HLA_name)|gene_assign.index.str.contains('SNPS_'+HLA_name)|gene_assign.index.str.contains('AA_'+HLA_name)]#print(gene_select.sort_values('pos').iloc[0],gene_select.sort_values('pos').iloc[-1])
    gene_assign[HLA_name][(gene_assign['pos']>=gene_select['pos'].min())&(gene_assign['pos']<=gene_select['pos'].max())]=1
    #txStart=gene_bed[gene_bed['name2']=='HLA-{}'.format(HLA_name)]['txStart']
    ##txEnd=gene_bed[gene_bed['name2']=='HLA-{}'.format(HLA_name)]['txEnd']
    #print(int(txStart),gene_assign_HLA[gene_assign_HLA[HLA_name]==1]['pos'].min(),gene_assign_HLA[gene_assign_HLA[HLA_name]==1]['pos'].max(),int(txEnd))     
"""
for idx,row in gene_bed.iterrows():
    exon_range_list=list(zip(row['exonStarts'].strip(',').split(','),row['exonEnds'].strip(',').split(',')))
    for start,end in exon_range_list:
        start=int(start);end=int(end)
        gene_assign_nonHLA_bool=((gene_assign_nonHLA['pos']>=start) & (gene_assign_nonHLA['pos']<=end))
        gene_assign_nonHLA[row['name2']][gene_assign_nonHLA_bool]=1        
"""            

In [9]:
phenotypes=pd.read_csv(pheno_all_file_path,sep='\t').set_index('ID')
binary_continuous_traits=sorted(phenotypes.columns[~phenotypes.columns.str.contains('x_ray')])

In [10]:
for phenotype_name in binary_continuous_traits:
    
    pheno=pd.read_csv(data_out_pheno_path+phenotype_name+'.phe',sep='\t',names=['FID','IID','pheno'])
    phenotype_type='binary' if len(pheno['pheno'].value_counts())<3 else 'continuous'
    data_out_assoc_phenotype_path=data_out_assoc_path+phenotype_name+'/'
    
    for step_idx in range(1,100):
        GAT_result=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.GAT.result.tsv'.format(step_idx),sep='\t')
        plink_result=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.PHENO2.glm.{}'.format(step_idx,'logistic' if phenotype_type=='binary' else 'linear'),sep='\t')
        print(phenotype_name,phenotype_type)
        
    
        plink_result_munge=plink_result[plink_result['TEST']=='ADD'].drop(columns='#CHROM').rename(columns={'TEST':'term','ID':'marker_name','BETA':'coef','SE':'std','T_STAT':'Z','OBS_CT':'nobs'})
        plink_result_munge['A2']=plink_result_munge.apply(lambda x: x['ALT'] if x['A1']==x['REF'] else x['REF'],axis=1)
        plink_result_munge=plink_result_munge.drop(columns=['REF','ALT'])
        plink_result_munge['note']='unphased bialleic'  
        result_merge=pd.concat([GAT_result,plink_result_munge])[['marker_name','note','term','POS','Z','std','chisq','coef','df','A1','A2','multi_allele','nobs','P']]
        
        result_merge_sorted=result_merge.sort_values('P')
        result_merge_sorted['step']=step_idx
        result_merge_sorted['gene']='a'
        break

    #'data/out_assoc/{}/step_01.plink.PHENO2.glm.linear'.format(i))
    
    
    break

ALP continuous


/home/ch6845/tools/miniconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [11]:
result_merge_sorted

,marker_name,note,term,POS,Z,std,chisq,coef,df,A1,A2,multi_allele,nobs,P
113652,6:30165273_C/T,unphased bialleic,ADD,30165273.0,-18.029400,0.525335,NaN,-9.471480,NaN,T,C,NaN,59469,1.786990e-72
106155,rs2523942,unphased bialleic,ADD,30033065.0,6.014320,2.019910,NaN,12.148400,NaN,G,T,NaN,59145,1.817020e-09
94234,6:29925086_T/G,unphased bialleic,ADD,29925086.0,5.878330,1.957390,NaN,11.506200,NaN,G,T,NaN,59469,4.166410e-09
305431,6:32537646_C/T,unphased bialleic,ADD,32537646.0,5.231500,0.473076,NaN,2.474900,NaN,C,T,NaN,59469,1.687100e-07
305557,6:32538123_G/A,unphased bialleic,ADD,32538123.0,5.172520,0.806447,NaN,4.171360,NaN,A,G,NaN,59469,2.317040e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107779,6:30060629_G/C,unphased bialleic,ADD,30060629.0,0.000073,0.475988,NaN,0.000035,NaN,C,G,NaN,59469,9.999420e-01
107737,6:30060364_G/A,unphased bialleic,ADD,30060364.0,0.000073,0.475988,NaN,0.000035,NaN,A,G,NaN,59469,9.999420e-01
107709,6:30060021_G/A,unphased bialleic,ADD,30060021.0,0.000073,0.475988,NaN,0.000035,NaN,A,G,NaN,59469,9.999420e-01
107877,6:30061202_G/A,unphased bialleic,ADD,30061202.0,0.000073,0.475988,NaN,0.000035,NaN,A,G,NaN,59469,9.999420e-01


# Load variables

In [4]:
gene_bed_path='data/known_genes_chr6.hg18.txt'

# Gene assignment

In [34]:
loci_info_list=[]
for phenotype_name in phenotype_list:
    #phenotype_name='ALP'
    print(phenotype_name)
    data_out_assoc_phenotype_path=data_out_assoc_path+phenotype_name+'/'
    num_independent=len(glob.glob(data_out_assoc_phenotype_path+'step_*.aa'))
    phenotype_data=pd.read_csv(data_out_assoc_phenotype_path+'phenotype.phe',sep='\t',header=None,names=['FID','IID','pheno'])
    phenotype_type= 'continuous' if len(phenotype_data['pheno'].unique())>3 else 'binary'
    
    for idx in range(1,num_independent+1):
        omnibus_assoc_result=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.omnibus.assoc.munged'.format(idx),sep='\t')
        plink_assoc_result=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.PHENO2.glm.{}.munged'.format(idx,'logistic' if phenotype_type=='binary' else 'linear'),sep='\t').iloc[:,1:]
        #print(plink_assoc_result)
        plink_cond_f=open(data_out_assoc_phenotype_path+'step_{:02d}.plink.cond'.format(5),'r')
        plink_cond=[cond.strip() for cond in plink_cond_f.readlines()]
        plink_cond_f.close()
        plink_covar=pd.read_csv(data_out_assoc_phenotype_path+'step_{:02d}.plink.covar'.format(1),sep='\t')
            
        omnibus_top=omnibus_assoc_result.sort_values('P').iloc[0]
        plink_top=plink_assoc_result.sort_values('P').iloc[0]
            
        assoc_type='plink' if plink_top['P']<omnibus_top['P'] else 'omnibus'
        
        variant=plink_top['ID'] if assoc_type=='plink' else  omnibus_top['Variant']
        pos=bim_aa.loc[plink_top['ID']]['pos'] if assoc_type=='plink' else  omnibus_top['Variant']
        p_val=plink_top['P'] if assoc_type=='plink' else omnibus_top['P']
        
        if assoc_type=='omnibus':
            gene_name='HLA_'+variant.split('_')[1]
        elif assoc_type=='plink':
            if variant[:3]=='HLA':
                gene_name='HLA_'+variant[4:].split('*')[0]
            else:
                # if the variant is itself in HLA transcription region
                if gene_assign_HLA.iloc[:,1:].loc[variant].sum()==1:
                    gene_name='HLA_'+gene_assign_HLA.iloc[:,1:].loc[variant].sort_values(ascending=False).index[0]
                elif gene_assign_HLA.iloc[:,1:].loc[variant].sum()>1:
                    raise
                # if the variant is itself in non-HLA gene coding region
                elif gene_assign_nonHLA.iloc[:,1:].loc[variant].sum()==1:
                    gene_name='HLA_'+gene_assign_nonHLA.iloc[:,1:].loc[variant].sort_values(ascending=False).index[0]
                elif gene_assign_nonHLA.iloc[:,1:].loc[variant].sum()>1:
                    raise
                else:             
                    r2_list=[]
                    for idx_bim,(SNP,row) in enumerate(bim_aa.iterrows()):
                        r2=pearsonr(plink_aa.get_geno_marker(variant),plink_aa.get_geno_marker(SNP))[0]**2
                        r2_list.append(r2)
                    r2_df=pd.DataFrame(r2_list,index=bim_aa.index)    
                    
                    gene_assign_HLA_select=gene_assign_HLA[r2_df[0]>0.5].iloc[:,1:]
                    gene_assign_nonHLA_select=gene_assign_nonHLA[r2_df[0]>0.7].iloc[:,1:]
                    # if the variant is in strong LD(>0.7) with a variant in HLA transcription region
                    if gene_assign_HLA_select.sum(axis=0).sum()==1: 
                        gene_name=gene_assign_HLA_select.sum(axis=0).sort_values(ascending=False).index[0]
                    elif gene_assign_HLA_select.sum(axis=0).sum()>1:
                        gene_assign_HLA_select_select_index=gene_assign_HLA_select.sum(axis=1).sort_values(ascending=False).index[gene_assign_HLA_select.sum(axis=1).sort_values(ascending=False)>0]
                        gene_assign_HLA_select_select_index_sorted=r2_df.loc[gene_assign_HLA_select_select_index].sort_values(0,ascending=False).index
                        if gene_assign_HLA_select.loc[gene_assign_HLA_select_select_index_sorted[0]].sum()==1:
                            gene_name=gene_assign_HLA_select.loc[gene_assign_HLA_select_select_index_sorted[0]].sort_values(ascending=False).index[0]
                        else:
                            raise
                    # if the variant is in moderate LD(>0.5) with a variant in non-HLA gene coding region
                    elif gene_assign_nonHLA_select.sum(axis=0).sum()==1:
                        gene_name=gene_assign_nonHLA_select.sum(axis=0).sort_values(ascending=False).index[0]
                    elif gene_assign_nonHLA_select.sum(axis=0).sum()>1:
                        gene_assign_nonHLA_select_select_index=gene_assign_nonHLA_select.sum(axis=1).sort_values(ascending=False).index[gene_assign_nonHLA_select.sum(axis=1).sort_values(ascending=False)>0]
                        gene_assign_nonHLA_select_select_index_sorted=r2_df.loc[gene_assign_nonHLA_select_select_index].sort_values(0,ascending=False).index
                        if gene_assign_nonHLA_select.loc[gene_assign_nonHLA_select_select_index_sorted[0]].sum()==1:
                            gene_name=gene_assign_nonHLA_select.loc[gene_assign_nonHLA_select_select_index_sorted[0]].sort_values(ascending=False).index[0]
                        else:
                            raise
                    #in other cases, assign to the nearest gene
                    else:
                        gene_bed_copy=gene_bed.copy()
                        gene_bed_copy['txStartDist']=(gene_bed_copy['txStart']-pos).abs()
                        gene_bed_copy['txEndDist']=(gene_bed_copy['txEnd']-pos).abs()
                        gene_bed_copy['txDist']=gene_bed_copy[['txStartDist','txEndDist']].min(axis=1)
                        gene_name=gene_bed_copy.sort_values('txDist').iloc[0]['name2']     
        
        
        loci_info={'phenotype_name':phenotype_name,
                   
                   'variant': variant,
                   'pos':pos,
                   'P':p_val,
                   
                   'Gene':gene_name,
                    'total':(phenotype_data['pheno']!=-9).sum(),
                   'case':(phenotype_data['pheno']==2).sum() if phenotype_type=='binary' else (phenotype_data['pheno']!=-9).sum(),
                   'control':(phenotype_data['pheno']==1).sum() if phenotype_type=='binary' else 0,
                   'A1/A2':((plink_top['A1'],(bim.loc[plink_top['ID']]['a2'] if plink_top['A1']!=bim.loc[plink_top['ID']]['a2'] else bim.loc[plink_top['ID']]['a1']))) if assoc_type=='plink' else ('-','-'),
                   'beta(se)': '{}({})'.format(plink_top['BETA'],plink_top['SE']) if assoc_type=='plink' else '-',
                   'chisq(df)':'{}({})({})'.format(omnibus_top['chisq'],omnibus_top['deltaDF'],','.join(omnibus_top['Residues'])) if assoc_type=='omnibus' else '-',
                   'conditional_covariate':plink_cond+plink_covar.columns.tolist()
                  }
        
        loci_info_list.append(loci_info)
        if idx==num_independent:
            if p_val>5e-8:
                print('error')

ALP


FileNotFoundError: [Errno 2] File b'data/out_assoc/ALP/step_05.omnibus.assoc.munged' does not exist: b'data/out_assoc/ALP/step_05.omnibus.assoc.munged'